In [3]:
 %load_ext autoreload


# From ZK to Bulletproofs

In [4]:
from klefki.types.algebra.concrete import EllipticCurveCyclicSubgroupSecp256k1 as Cruve
from klefki.types.algebra.concrete import EllipticCurveGroupSecp256k1 as ECG
from klefki.types.algebra.concrete import FiniteFieldCyclicSecp256k1 as CF

G = Cruve.G

## 2.3 A Pedersen commitment in elliptic curve form

In [5]:
import random

N = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141
def random_cf() -> CF: return CF(random.randint(1, N))

$$C=rH+aG$$

$H$ is a curve point, for which nobody knows the discrete logarithm $q$ s.t. $H = qG$.

In [6]:
q = random_cf()
H = G@q

In [7]:
def C(r: CF, a: CF): return (H@r + G@a)

### Homomorphism

In [8]:
r_1, a_1, r_2, a_2 = random_cf(), random_cf(), random_cf(), random_cf()

In [9]:
assert C(r_1, a_1) + C(r_2, a_2) == H@r_1 + G@a_1 + H@r_2 + G@a_2

In [10]:
assert C(r_1, a_1) + C(r_2, a_2) == C(r_1 + r_2, a_1 + a_2)

### The Vector Pedersen Commitment

To extend to a more powerful form of the Pedersen commitment already deﬁned, we go from:
    $$C = rH + aG$$
    to:
        $$C=rH + \sum_{i=0}^n v_iG_i = rH + \mathbf{v}\mathbf{G}$$

The individual G i s can be constructed using a simple algorithm of the form already mentioned (like, take a $\mathcal{H}(encode(G)||i)$ where $\mathcal{H}$ represents some hash function).

In [72]:
from klefki.types.algebra.utils import encode, decode
from hashlib import sha256

In [82]:
G_is = list(
    map(decode,
        (map(lambda x: sha256(x.encode()).hexdigest(), encode(G)))
       )
)

### Knowledge of a set of vectors

Our argument of knowledge will come after we have generated a set of com- mitments for each of m vectors $x_1 , x_2 , \cdots , x_m$ , each of the same dimension $N(\neq m)$).Explicitly:

$$
C_1=r_1H + x_1G\\
C_2=r_2H + x_2G\\
\vdots\\
C_m=r_mH+x_mG\\
$$

In [134]:
m = 128
x = [random_cf() for i in range(0, m)]
r = [random_cf() for i in range(0, m)]
q = random_cf()
H = G@q
C = list(map(lambda x: H @ x[0] + G @ x[1],zip(r, x)))

#### The process:

1. $P\rightarrow V: C_0$ (0 (a new commitment to a newly chosen random vector of dimension $N$)

2. $V\rightarrow P: e$ (a random scalar)

3. $P\rightarrow V: (\mathbf{z}, s)$(a single vector of dimension N, and another scalar)

$$
z=\sum_{i=0}^m e^ix_i,\ s=\sum_{i=0}^m e^ir_i
$$

Note that the summations start at 0; this means that the sums include the extra, random commitment, indexed 0, that was created as the ﬁrst step of the interaction.

In [135]:
from functools import reduce
P, V = {}, {}

# Step 1
P['C_0'] = G @ random_cf()
V['C_0'] = P['C_0']

# Step 2
V['e'] = random_cf()
P['e'] = V['e']

# Step 3
P['z'] = reduce(lambda x, y: x+y, [x[i] @ (e ** (i+1)) for i in range(0, m)])
P['s'] = reduce(lambda x, y: x+y, [r[i] @ (e ** (i+1)) for i in range(0, m)])
V['z'], V['s'] = P['z'], P['s']


Having received this $(z, s)$, the verifer of course needs to verify whether the proof is valid. He does the following:

$$
\sum_{i=0}^m e^iC_i \stackrel{?}{=} sH+\mathbf{z}\mathbf{G}
$$

In [139]:
assert reduce(lambda x, y: x+y, [C[i] @ (V['e'] ** (i+1)) for i in range(0, m)]) == H @ s + G @ z

AssertionError: 

### Schnorr’s identity protocol

Prover starts with a public key P and a corresponding private key $x$ s.t. $P = xG$.
Prover wishes to prove in zero knowledge, that he knows $x$.

1. $P\rightarrow V:xG$
2. $P\rightarrow V:R$ (a new random curve point, but $P$ knows $k$ s.t. $R=kG$)
3. $V\rightarrow P:e$ (a random scalar)
4. $P\rightarrow V:s$ (which $P$ calcuated from the quation $s=k+ex$)

Note: the transcript referred to above, would here be: $(R, e, s)$.

In [70]:
P, V = {}, {}
x = random_cf()
# step 1
P['x'] = x
V['P'] = G @ P['x']

# step 2
P['k'] = random_cf()
V['R'] = G @ P['k']

# step 3
V['e'] = random_cf()
P['e'] = V['e']

# step 4
P['s'] = P['k'] + P['e'] * P['x']
V['s'] = P['s']

transcript = V['R'], V['e'], V['s']

Veriﬁcation works fairly trivially: veriﬁer checks $sG \stackrel{?}{=} R+eP$.


In [71]:
assert G @ V['s'] == V['R'] + V['P'] @ V['e']


The “Simulator”, which controls the execution of the veriﬁer, given the public key P, just as the Veriﬁer would be, can fake a valid transcript as follows:

Choose $s$ randomly. Then, choose $e$, also randomly. Finally, we only need to choose $R$ to create a complete conversation transcript; it must be $R = sG − eP$. 

$$
R=sG-eP\\
kG=sG-eP\\
sG=kG+eP\\
sG=kG+xeG\\
s=k+ex\\
$$

Then we have successfully simulated a conversation which is entirely valid: $(R, e, s)$, without ever knowing the secret key $x$, and which it’s easy to see is randomly distributed in the same way as a real one would be ($R$ is a free variable).

In [66]:
s, e = random_cf(), random_cf()
R = G @ s - V['P'] @ e

In [69]:
assert G @ s == R + V['P'] @ e

### vector proof of knowledge